# Sprawozdanie ze współbieżnej eliminacji metodą Gaussa

## Franciszek Job, 16.12.2024

<!-- TOC -->

- [1. Część teoretyczna](#1-czesc-teoretyczna)
- [2. Implementacja](#2-implementacja)

<!-- /TOC -->

## 1. Część teoretyczna

### Macierz

Załóżmy, że mamy następujący układ równań:

$$
\begin{bmatrix}
    M_1_1 & M_1_2 & M_1_3 & ... & M_1_n \\
    M_2_1 & M_2_2 & M_2_3 & ... & M_2_n \\
    M_3_1 & M_3_2 & M_3_3 & ... & M_3_n \\
    ... & ... & ... & ... & ... \\
    M_n_1 & M_n_2 & M_n_3 & ... & M_n_n \\
\end{bmatrix}

\begin{bmatrix}
    x_1 \\
    x_2 \\
    x_3 \\
    ... \\
    x_n \\
\end{bmatrix}

=

\begin{bmatrix}
    y_1 \\
    y_2 \\
    y_3 \\
    ... \\
    y_n \\
\end{bmatrix}

$$

Głównie będziemy się niżej zajmować macierzą $M$.

### Niepodzielne operacje

Teraz możemy zdefiniować niepodzielne operacje na jej elementach.

$A_k_i$ -> $M_k_i / M_i_i = m_k_i$  (dzielenie elementu z $k$-tego wiersza przez element z $i$-tego wiersza, w kolumnie $i$)

<!-- TODO: Zapytac czy kolumna j jest ok -->
$B_k_j_i$ -> $M_i_j * m_k_i = n_k_j_i$ (mnożenie elementu z $i$-tego wiersza przez $m_k_i$ w kolumnie $j$)

$C_k_j_i$ -> $M_k_j - n_k_j_i = M_k_j$ (odejmowanie $n_k_j_i$ od elementu z $k$-tego wiersza w kolumnie $j$)

### Alfabet (w sensie teorii śladów)

Następnie definijemy sobie alfabet (w sensie teorii śladów). Jest on sumą trzech alfabetów, odpowiednio dla operacji $A$, $B$ i $C$.

W takim razie:

$ \Sigma_{A} = \{ A_k_i \} $ takie, że $ i \in \{ 1, ..., n - 1\} $ oraz $ k \in \{ i+1, ..., n - 1 \} $

$ \Sigma_{B} = \{ B_k_j_i \} $ takie, że $ i \in \{ 1, ..., n - 1\} $ oraz $ j \in \{ i, ..., m \} $ oraz $ k \in \{ i+1, ..., n - 1 \} $

$ \Sigma_{C} = \{ C_k_j_i \} $ takie, że $ i \in \{ 1, ..., n - 1\} $ oraz $ j \in \{ i, ..., m \} $ oraz $ k \in \{ i+1, ..., n - 1 \} $

Cały alfabet:

$ \Sigma = \Sigma_{A} \cup \Sigma_{B} \cup \Sigma_{C} $

### Relacje zależności/niezależności

Definiujemy relację zależności i niezależności:

$ D_1 = (A_k_i, B_k_j_i) $ <br>
$ D_2 = (B_k_j_i, C_k_j_i) $ <br>
$ D_3 = (A_k_{c}_j_{c}_i_{c}, A_k_{a}_i_{a}) $ <br>
$ D_4 = (A_k_{c}_j_{c}_i_{c}, B_k_{b}_j_{b}_i_{b}) \wedge k_c = i_b $ <br>
$ D_5 = (C_k_j_i_{C1}, C_k_j_i_{C2}) $ <br>
$ D = sym(D_1 \cup D_2 \cup D_3 \cup D_4 \cup D_5)^+ \cup I_{\Sigma} $
$ I = \Sigma^2 - D $


### Algorytm eleminiacji metodą Gaussa

Idea jest taka, że algorytm to tak naprawdę ciąg podciągów, gdzie każdy podciąg ma rerezentuje przemnożenie i odjęcie $i$-tego wiersza od $k$-tego wiersza (co oczywiście prowadzi do wyzerowania). Każdy taki podciąg ma postać:

$ c_k_i = A_k_i, B_k_i_i, C_k_i_i,..., B_k_m_i, C_k_m_i $

Cały więc algorytm to po prostu:

$ c_2_1, ..., c_n_1, c_3_2, ..., c_n_2, ..., c_n_{n-1} $

### Graf Diekerta

![Graf Diekerta dla przykładu 1](diekert.png)

### Postać normalna Foaty
Wiemy, że $i = 1, ..., n - 1 $, a więc możemy zdefiniować:

$F_{A_{i}} = \{A_{k,i}: k=i+1,\dots,n \}$

<br>

$F_{B_{i}} = \{B_{k,j,i}: j = i, \dots, m;\; k = i+1, \dots, n \}$

<br>
$F_{C_{i}} = \{C_{k,j,i}: j = i, \dots, m;\; k = i+1, \dots, n \}$

<br>

Dalej, postać normalna Foaty to:

$ [F_{A_{1}}][F_{B_{1}}][F_{C_{1}}][F_{A_{2}}][F_{B_{2}}][F_{C_{2}}]\dots[F_{A_{n-1}}][F_{B_{n-1}}][F_{C_{n-1}}] $

<br />

## 2. Implementacja

W tej części zajmiemy się już faktyczną implementacją algorytmu eliminacji metodą Gaussa.

Przykład pliku wejściowego:

```
3
2.0 1.0 3.0
4.0 3.0 8.0
6.0 5.0 16.0
6.0 15.0 27.0
```

Zakładamy, że ostatni wiersz to wektor wyrazów wolnych.

Funkcja do wczytania pliku

In [1]:
def read_input_file(file_name):
    """Wczytuje wymiar i macierz rozszerzoną M z pliku."""
    with open(file_name, 'r') as file:
        dim = int(file.readline())

        matrix = []
        for i in range(dim):
            matrix.append(list(map(float, file.readline().split())))

        rhs = list(map(float, file.readline().split()))
        for i in range(dim):
            matrix[i].append(rhs[i])

        return dim, matrix

Funkcje odpowiadające operacjom

In [2]:
def compute_A(matrix, k, i):
    """Operacja A_k_i: Obliczenie współczynnika skalującego m[k][i]."""
    return matrix[k][i] / matrix[i][i]


def compute_B(matrix, m, i, k, j):
    """Operacja B_k_j_i: Obliczenie wartości pośredniej n[k][j][i]."""
    return matrix[i][j] * m


def compute_C(matrix, k, j, n):
    """Operacja C_k_j_i: Aktualizacja wartości macierzy."""
    matrix[k][j] -= n

Funkcje obliczające w obrębie klasy Foaty, wykorzystujące pule wątków

In [3]:
from concurrent.futures import ThreadPoolExecutor


def execute_FA_parallel(matrix, pivot_row, dim):
    """Klasa Foaty FA: Obliczanie współczynników skalujących z równoległością."""
    m = [0] * dim

    def compute(k):
        m[k] = compute_A(matrix, k, pivot_row)

    with ThreadPoolExecutor() as executor:
        executor.map(compute, range(pivot_row + 1, dim))

    return m


def execute_FB_parallel(matrix, m, pivot_row, dim):
    """Klasa Foaty FB: Obliczanie wartości pośrednich z równoległością."""
    n = [[0] * (dim + 1) for _ in range(dim)]

    def compute(k, j):
        n[k][j] = compute_B(matrix, m[k], pivot_row, k, j)

    with ThreadPoolExecutor() as executor:
        # Tworzymy pary (k, j) dla każdej kombinacji wierszy i kolumn
        executor.map(lambda args: compute(*args),
                     ((k, j) for k in range(pivot_row + 1, dim)
                              for j in range(pivot_row + 1, dim + 1)))

    return n


def execute_FC_parallel(matrix, n, pivot_row, dim):
    """Klasa Foaty FC: Aktualizacja wartości macierzy z równoległością."""
    def update(k, j):
        compute_C(matrix, k, j, n[k][j])

    with ThreadPoolExecutor() as executor:
        # Tworzymy pary (k, j) dla każdej kombinacji wierszy i kolumn
        executor.map(lambda args: update(*args),
                     ((k, j) for k in range(pivot_row + 1, dim)
                              for j in range(pivot_row + 1, dim + 1)))

Funkcja eliminacji Gaussa

In [4]:
def gauss_elimination_parallel(matrix, dim):
    """Eliminacja Gaussa z równoległością: Redukuje macierz do postaci trójkątnej górnej."""
    for pivot_row in range(dim - 1):
        # Klasa FA: Obliczamy współczynniki skalujące
        m = execute_FA_parallel(matrix, pivot_row, dim)

        # Klasa FB: Obliczamy wartości pośrednie
        n = execute_FB_parallel(matrix, m, pivot_row, dim)

        # Klasa FC: Aktualizujemy macierz
        execute_FC_parallel(matrix, n, pivot_row, dim)

Funkcja rozwiązująca układ równań

In [5]:
def backward_substitution(matrix, dim):
    """Rozwiązuje układ równań po eliminacji Gaussa."""
    
    # Tablica wyników
    x = [0.0] * dim

    # Iterujemy od ostatniego wiersza do pierwszego
    for i in range(dim - 1, -1, -1):
        # Rozpoczynamy od wyrazu wolnego (ostatnia kolumna)
        rhs = matrix[i][dim]

        # Odejmujemy iloczyny elementów macierzy i wcześniejszych wyników x
        for j in range(i + 1, dim):
            rhs -= matrix[i][j] * x[j]

        # Dzielimy przez współczynnik na głównej przekątnej
        x[i] = rhs / matrix[i][i]

    return x


Funkcja główna

In [6]:
def gauss_solver(input_file, output_file):
    # Wczytujemy macierz rozszerzoną M
    dim, matrix = read_input_file(input_file)

    # Eliminacja Gaussa
    gauss_elimination_parallel(matrix, dim)

    # Rozwiązanie układu
    solution = backward_substitution(matrix, dim)

    write_output_file(output_file, dim, solution)

Funkcja do zapisu wyniku

In [7]:
def write_output_file(output_file, dim, solution):
    """Zapisuje wynik eliminacji Gaussa do pliku wyjściowego."""
    with open(output_file, 'w') as file:
        file.write(f"{dim}\n")
        print(f"{dim}\n")

        for i in range(dim):
            row = [f"{1.0 if i == j else 0.0:.1f}" for j in range(dim)]
            file.write(" ".join(row) + "\n")
            print(" ".join(row) + "\n")

        file.write(" ".join([f"{x}" for x in solution]) + "\n")
        print(" ".join([f"{x}" for x in solution]) + "\n")

Przykładowe użycie

In [8]:
test_cases = (
    ("input.txt", "output.txt"),
    ("input2.txt", "output2.txt"),
)

for i in range(len(test_cases)):
    print(f"Test case {i + 1}:")
    gauss_solver(*test_cases[i])

Test case 1:
3

1.0 0.0 0.0

0.0 1.0 0.0

0.0 0.0 1.0

1.0 1.0 1.0

Test case 2:
15

1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0

0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0

